Plan: use qupath_annotations_latest instead of TissueAnnotator outputs for loading -- consistency across datasets and easier to use.
1. Load only information relevant to the task at hand from json
2. Load npy files for each image from dir, with option to create
3. Register the dataset
4. Idk

In [7]:
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.config import get_cfg
from detectron2.engine import default_argument_parser, default_setup, launch
from typing import Dict, Set, List, Tuple, Iterator
import os
import logging
import json
import copy

In [8]:
train_sample = '/mnt/RSX/Datasets_pathology/SRI_OSCC_lymph_train'

In [9]:
def find_anno_dir(parent_dir: str) -> List[str]:
    """
    Find qupath exported annotations directory
    """
    
    if os.path.exists(os.path.join(parent_dir, 'xupath_annotations_latest')):
        return os.path.join(parent_dir, 'qupath_annotations_latest')
    else:
        anno_dirs = []
        for root, dirs, files in os.walk(parent_dir):
            for d in dirs:
                if 'annotations' in d:
                    anno_dirs.append(os.path.join(root, d))
        # user chooses if there are multiple annotation folders
        print('Found multiple annotation folders:')
        for i, anno_dir in enumerate(anno_dirs):
            print(f'{i}: {os.path.relpath(anno_dir, parent_dir)}')
        choice = input('Choose annotation folder index')
        if choice.isdigit() and int(choice) < len(anno_dirs):
            return anno_dirs[int(choice)]    
        else:
            raise ValueError('Annotation folder not found')

In [10]:
def search_recursive(d: Dict, key: str) -> Iterator:
        """Helper function for finding which level of json annotations has
        the matching key.
        """
        for k, v in d.items():
            if isinstance(v, Dict):
                for match in search_recursive(v, key):
                    yield match
            if k == key:
                # generator function - saves in memory until called
                # (use for loop to call)
                yield v

In [11]:
find_anno_dir(train_sample)

Found multiple annotation folders:
0: qupath_annotations_latest
1: qupath_annotations_V4
2: qupath_annotations_V3
3: qupath_annotations_V2
4: qupath_annotations_V1


'/mnt/RSX/Datasets_pathology/SRI_OSCC_lymph_train/qupath_annotations_latest'

In [12]:
def parse_json_by_task(json_file):
    """
    Parse json file by task
    """
    with open(json_file, 'r') as f:
        data = json.load(f)
        return data

In [98]:
json_file = '/mnt/RSX/Datasets_pathology/SRI_OSCC_lymph_train/qupath_annotations_latest/Case 1 G7.json'
data = parse_json_by_task(json_file)
tissue_types = ['lymph', 'non-lymph']
tissue_data = []
for i in data:
    if any(tissue in list(search_recursive(i, 'name')) for tissue in tissue_types):
        tissue_data.append(i)

In [111]:

## scale bbox, convert to xyxy from [x1y1, x2y1, x2y2, x1y2] 
# rename key to bbox

def scale_bboxes_qupath(anno, ref_dim, target_dim):
    x_scale = ref_dim[1] / target_dim[1]
    y_scale = ref_dim[0] / target_dim[0]
    for i in anno:
        [coords] = i['coordinates']
        # First, build XYXY
        x0 = int(coords[0][0] / x_scale)
        y0 = int(coords[0][1] / y_scale)
        x1 = int(coords[2][0] / x_scale)
        y1 = int(coords[2][1] / y_scale)
        i['bbox'] = [x0, y0, x1, y1]
        del i['coordinates']
        # make it as 'bbox': asdjhkf
    return anno
        
#scale_bboxes_qupath(anno_info, [2,2], [1,1])  
    

In [125]:
# get coordinates for all relevant tissues and create boxes

def get_boxes(json_file, ref_dim, target_dim, tissue_types):
    
    with open(json_file, 'r') as f:
        data = json.load(f)
    tissue_data = []
    for i in data:
        if any(tissue in list(search_recursive(i, 'name')) for tissue in tissue_types):
            tissue_data.append(i)
    cat_map = {tissue: i for i, tissue in enumerate(tissue_types)}
    coords = []
    for k in tissue_data:
        ## add names to k 
        k['geometry']['category_id'] = cat_map[next(search_recursive(k, 'name'))]
        del k['geometry']['type']
        k['geometry']['bbox_mode'] = 0
        coords.append(next(search_recursive(k, 'geometry')))
    
    out = scale_bboxes_qupath(coords, ref_dim, target_dim)
    return coords, cat_map

In [126]:
annotation_dicts, cat_map = get_boxes(json_file, ref_dim, target_dim, tissue_types)

([{'category_id': 1, 'bbox_mode': 0, 'bbox': [7350, 794, 19541, 13678]},
  {'category_id': 1, 'bbox_mode': 0, 'bbox': [814, 5432, 10952, 16382]}],
 {'lymph': 0, 'non-lymph': 1})

In [26]:
+## get to detectron2 coco format

## [{'file_name': '/mnt/RSX/Datasets_pathology/GT_2023/TissueFinderV2/Case 9 M3.npy', 
# 'height': 1649, 
# 'width': 2560, 
# 'image_id': 'Case 9 M3',
# 'annotations': [{'category_id': 0, 'bbox': [97, 222, 1190, 1558], 'bbox_mode': 0}, {'category_id': 0, 'bbox': [1407, 435, 2410, 1629], 'bbox_mode': 0}]}, 

In [27]:
def get_coco_format(parent_dir, json_file, tissue_types):
    """
    Get coco format for detectron2
    """
    ## Determine image format
    fmt = os.path.splitext(os.listdir(parent_dir)[0])[1]
    print(fmt)
    info = get_boxes(json_file, tissue_types)
    for i in info:
        
    # coco_format = []engine
    

In [23]:
get_coco_format(train_sample, json_file, tissue_types)

.svs


TypeError: list indices must be integers or slices, not str